This file contains the heart disease data

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [52]:
original_data <- read_table2("data/cleveland.data",
                col_names = F)

#adding an index column to the data frame for future reference
data <- original_data %>%
        mutate(index = as.numeric(rownames(original_data))) %>%
        filter(index <= 2820) #excluding data after the 2820th row, as the data is unusable because there is too much missing data

# creates an individual data frame for each section of data
section_1 <- data %>%
            filter((index - 1) %% 10 == 0) %>%
            select(-X8)
colnames(section_1) <- c("a","b","c","d","e","f","g","index") #testing how to change names
section_2 <- data %>%
            filter((index - 1) %% 10 == 1) %>%
            mutate(index = index - 1)
section_3 <- data %>%
            filter((index - 1) %% 10 == 2) %>%
            mutate(index = index - 2)
section_4 <- data %>%
            filter((index - 1) %% 10 == 3) %>%
            mutate(index = index - 3)
section_5 <- data %>%
            filter((index - 1) %% 10 == 4) %>%
            mutate(index = index - 4)
section_6 <- data %>%
            filter((index - 1) %% 10 == 5) %>%
            mutate(index = index - 5)
section_7 <- data %>%
            filter((index - 1) %% 10 == 6) %>%
            mutate(index = index - 6)
section_8 <- data %>%
            filter((index - 1) %% 10 == 7) %>%
            mutate(index = index - 7)
section_9 <- data %>%
            filter((index - 1) %% 10 == 8) %>%
            mutate(index = index - 8)
section_10 <- data %>%
            filter((index - 1) %% 10 == 9) %>%
            mutate(index = index - 9) %>%
            select(-X6,-X7,-X8)

#combines all of the individual data tables into one data table
heart_data <- merge(section_1, section_2, by="index") %>%
            merge(section_3, by = "index") %>%
            merge(section_4, by = "index") %>%
            merge(section_5, by = "index") %>%
            merge(section_6, by = "index") %>%
            merge(section_7, by = "index") %>%
            merge(section_8, by = "index") %>%
            merge(section_9, by = "index") %>%
            merge(section_10, by = "index") %>%
heart_data

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_character(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double()
)

Warning message:
“766 parsing failures.
row col  expected    actual                  file
 10  -- 8 columns 5 columns 'data/cleveland.data'
 11  -- 8 columns 7 columns 'data/cleveland.data'
 20  -- 8 columns 5 columns 'data/cleveland.data'
 21  -- 8 columns 7 columns 'data/cleveland.data'
 30  -- 8 columns 5 columns 'data/cleveland.data'
... ... ......... ......... .....................
See problems(...) for more details.
”
Warning message in merge.data.frame(., section_5, by = "index"):
“column names ‘X1.x’, ‘X2.x’, ‘X3.x’, ‘X4.x’, ‘X5.x’, ‘X6.x’, ‘X7.x’, ‘X8.x’, ‘X1.y’, ‘X2.y’, ‘X3.y’, ‘X4.y’, ‘X5.y’, ‘X6.y’, ‘X7.y’, ‘X8.y’ are duplicated in the result”
Warning message in merge.data.frame(., section_6, by = "index"):
“column names ‘X1.x’, ‘X2.x’, ‘X3.x’, ‘X4.x’, ‘X5.x’, ‘X6.

ERROR: Error in heart_data(.): could not find function "heart_data"
